In [3]:
import numpy as np
import pandas as pd 
import os 
from astropy import units as u
from astropy.cosmology import FlatLambdaCDM
parent_dir=os.path.dirname(os.getcwd())

def read_line_emission_csv(path_line_emission_csv):
    """
    Read the csv file in which are stored the line emission's rest frequency.
    
    Parameter: 
    path_line_emission_csv (str): Path to file.csv within there are the line emission's rest frequency.
    
    Return:
    pd.DataFrame : Dataframe with line names and rest frequencies.
    """
    db_line = pd.read_csv(path_line_emission_csv, sep = ";")
    return db_line

def sed_reading(type_, path):
    cosmo = FlatLambdaCDM(H0=70 * U.km / U.s / U.Mpc, Tcmb0=2.725 * U.K, Om0=0.3)
    if type_ == "extended":
        file_path = path + "/sed_low_z_warm_star_forming_galaxy.dat"
        redshift = 10**(-4)
    elif type_ == "point":
        file_path = path + "/sed_low_z_type2_AGN.dat"
        redshift = 0.05
    else:
        return "Not valid type"
    
    try: 
        SED= pd.read_csv(file_path, sep="\s+")
        rename_columns = {
            'um' : 'GHz',
            'erg/s/Hz': 'Jy',
        }
        SED.rename(columns=rename_columns, inplace=True)
        SED['GHz']=SED['GHz'].apply(lambda x: (x* U.um).to(U.GHz, equivalencies=U.spectral()).value)
        if type_ == 'point': 
            SED['Jy']=SED['Jy']/((10.**(-26.))*(10.**7.)*4.*np.pi*(cosmo.luminosity_distance(redshift).value*(3.086e+22))**2.)*(3.846e+33*1e+10)
        else:
            SED['Jy']=SED['Jy']/((10.**(-26.))*(10.**7.)*4.*np.pi*(cosmo.luminosity_distance(redshift).value*(3.086e+22))**2.)*(3.846e+33*1e+9)
        return SED
    except FileNotFoundError:
         return "File not Found"

def process_spectral_data(type,redshift, central_frequency, delta_freq, source_frequency, n_lines=None):
    """
    Process spectral data based on the type of source, wavelength conversion,
    line ratios, and given frequency bands.
    
    Prameters:
    
    redshift: Redshift value to adjust the spectral lines and continuum.
    central_frequency: Central frequency of the observation band (GHz).
    delta_freq: Bandwidth around the central frequency (GHz).
    source_frequency: Frequency of the source obtained from metadata (GHz).
    lines: Optional list of line names provided by the user.
    n_lines: Number of additional lines to consider if lines is None.

    Output:


    """
    # Define the frequency range based on central frequency and bandwidth
    freq_min = central_frequency - delta_freq / 2
    freq_max = central_frequency + delta_freq / 2
    # Example data: Placeholder for continuum and lines from SED processing
    SED=sed_reading(type,os.path.join(parent_dir,'brightnes'))

    # Placeholder for line data: line_name, observed_frequency (GHz), line_ratio, line_error
    db_line = read_line_emission_csv(os.path.join(parent_dir,'brightnes','Calibrations_FIR(GHz).csv'))
    rest_frequencies = []
    for line_name in line_names:
        if line_name not in db_line["Line"].to_list():
            raise ValueError(f"The line name, '{line_name}', cannot be found in the dataframe.")
        rest_frequency = {}
        rest_frequency['line']=line_name
        rest_frequency['frequency(GHz)'] = db_line[db_line["Line"]==line_name]["freq(GHz)"].to_list()
        # rest_frequency = db_line[db_line["Line"]==line_name]["freq(GHz)"]
        rest_frequencies.append(rest_frequency)
    rest_frequencies =pd.DataFrame(rest_frequencies)
    rest_frequencies = rest_frequencies.explode('frequency(GHz)')
    line_names = rest_frequencies['line'].unique().tolist()
    for line in line_names:
        rest_frequency = rest_frequencies[rest_frequencies['line']==line]['frequency(GHz)'].tolist()
        print(f"Rest frequency/ies of {line}: {rest_frequency}\n")
    line_ratio=np.exp(c1-c2)
    line_error=
    # Shift the continuum and line frequencies by (1 + redshift)
    SED['GHz'] *= (1 + redshift)
    db_line['freq(GHz)'] = db_line['freq(GHz)'] * (1 + redshift)

    # Filter the continuum and lines to only include those within the frequency range
    continuum_mask = (SED['GHz'] >= freq_min) & (SED['GHz'] <= freq_max)
    filtered_continuum_frequencies = SED[continuum_mask]

    line_mask = (db_line['freq(GHz)'].astype(float) >= freq_min) & (db_line['freq(GHz)'].astype(float) <= freq_max)
    filtered_lines = db_line[line_mask]
    if len(filtered_lines) == 0:
        print("Warning: No valid frequencies provided by the user. Returning possible lines.")
    print(filtered_lines)
    #Determine which lines to output based on user input or nearest lines to source frequency
    if line_names is not None:
        # Filter out the lines that are not possible
        possible_lines = filtered_lines[np.isin(filtered_lines[:, 0], lines)]
        if len(possible_lines) == 0:
            print("Warning: No valid lines provided by the user. Returning possible lines.")
            possible_lines = filtered_lines
    else:
        # If no lines are provided, select the nearest line to the source frequency and additional lines if specified
        distances = np.abs(filtered_lines[:, 1].astype(float) - source_frequency)
        nearest_index = np.argmin(distances)
        if n_lines is None or n_lines > len(filtered_lines):
            n_lines = len(filtered_lines)
        nearest_lines = filtered_lines[np.argsort(distances)[:n_lines]]
        possible_lines = nearest_lines

    #Output the processed arrays and line information
    return filtered_continuum_frequencies, possible_lines[:, 0], possible_lines


In [6]:

# Example call to the function with hypothetical parameters
continuum_freqs, line_names = process_spectral_data(
    redshift=0.1, 
    central_frequency=46500, 
    delta_freq=7500, 
    source_frequency=48500
)

print("Filtered Continuum Frequencies:", continuum_freqs)
print("Line Names:", line_names)


         Line     freq(GHz)     c  err_c     d  err_d     a     b  disp  \
0         PAH  48353.622258 -2.20   0.36  0.00   0.00  0.00  0.00  0.00   
1     [SiVII]  46121.916615  0.00   0.00  0.00   0.00  0.83 -3.55  0.37   
2       H_{2}  43385.305065 -3.97   0.39  0.00   0.00  0.80 -2.40  0.34   
3      [ArII]  42950.208883 -3.96   0.32  0.00   0.00  0.84 -4.21  0.64   
4      [NeVI]  39188.556601  0.00   0.00  0.00   0.00  0.79 -1.48  0.42   
5         PAH  38934.085455 -1.64   0.36  0.00   0.00  0.00  0.00  0.00   
6       [ArV]  37948.412405  0.00   0.00  0.00   0.00  0.87 -3.85  0.32   
7         PAH  34859.588140 -2.16   0.36  0.00   0.00  0.00  0.00  0.00   
8     [ArIII]  33347.325695 -4.22   0.69  0.00   0.00  0.98 -4.15  0.37   
9       H_{2}  31034.415942 -3.96   0.52  0.00   0.00  1.07 -5.32  0.34   
10      [SIV]  28578.880648 -3.95   0.69  0.00   0.00  0.90 -2.96  0.24   
11        PAH  26648.218489 -2.29   0.36  9.01   0.28  0.00  0.00  0.00   
12      [CaV]  26114.3256

In [ ]:

# Example call to the function with hypothetical parameters
continuum_freqs, line_names, line_info = process_spectral_data(
    redshift=0.1, 
    central_frequency=5, 
    delta_freq=1, 
    source_frequency=4.85
)

print("Filtered Continuum Frequencies:", continuum_freqs)
print("Line Names:", line_names)
print("Line Information:", line_info)
